In [1]:
# Import Libary

from numpy import mat, cov, mean, hstack, multiply,sqrt,diag, genfromtxt, \
squeeze, ones, array, vstack, kron, zeros, eye, savez_compressed
from numpy.linalg import lstsq, inv
from scipy.stats import chi2
from pandas import read_csv

# Import Data

In [2]:
# Import Data
data = read_csv('..\Data\FamaFrench.csv')

In [3]:
# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['VWMe', 'SMB', 'HML']].values
riskfree = data['RF'].values
portfolios = data.ix[:, 5:].values

In [4]:
# Use mat for easier linear algebra
factors = mat(factors)
riskfree = mat(riskfree)
portfolios = mat(portfolios)

In [5]:
# Shape information
T,K = factors.shape
T,N = portfolios.shape

In [6]:
# Reshape rf and compute excess returns
riskfree.shape = T,1
excessReturns = portfolios - riskfree

# Fama McBeth

In [7]:
# Time series regressions
X = hstack((ones((T, 1)), factors))
out = lstsq(X, excessReturns)
alpha = out[0][0]
beta = out[0][1:]

In [9]:
# Crosssection regression
avgExcessReturns = mean(excessReturns, 0)

out = lstsq(beta.T, avgExcessReturns.T)
riskPremia = out[0]

In [10]:
beta

matrix([[ 1.30989756,  1.08529063,  1.074662  ,  0.96302219,  0.98496718,
          1.06909171,  1.04155003,  0.95902319,  0.97875629,  1.05020197,
          1.14164406,  1.0132696 ,  1.01286868,  0.96148018,  1.14467854,
          1.06612416,  1.03078498,  1.00960431,  1.04365536,  1.22842095,
          1.03104193,  0.95757439,  0.97527731,  1.05455022,  1.10451275],
        [ 1.28916295,  1.61001141,  1.18117487,  1.22485887,  1.34530808,
          1.05198817,  0.98802256,  0.86189893,  0.81778888,  0.93732422,
          0.7883002 ,  0.51512103,  0.41299482,  0.46464673,  0.49699993,
          0.28566577,  0.24300243,  0.22141899,  0.20155396,  0.29737316,
         -0.1507105 , -0.18934339, -0.21726041, -0.1731627 ,  0.00759986],
        [ 0.39425156,  0.33173673,  0.46480276,  0.5853827 ,  0.90516173,
         -0.26472086,  0.18770969,  0.35532098,  0.55624082,  0.84930135,
         -0.19804113,  0.07200581,  0.33794592,  0.50675635,  0.9142738 ,
         -0.36923371,  0.13278923,  

In [11]:
riskPremia

matrix([[ 0.55535352],
        [ 0.2394288 ],
        [ 0.23400198]])

# Inference

In [14]:
# Moment conditions
X = hstack((ones((T, 1)), factors))
p = vstack((alpha, beta))
epsilon = excessReturns - X * p
moments1 = kron(epsilon, ones((1, K + 1)))
moments1 = multiply(moments1, kron(ones((1, N)), X))
u = excessReturns - riskPremia.T * beta
moments2 = u * beta.T

In [15]:
# Score covariance
S = mat(cov(hstack((moments1, moments2)).T))

In [18]:
# Jacobian
G = mat(zeros((N * K + N + K, N * K + N + K)))
SigmaX = X.T * X / T
G[:N * K + N, :N * K + N] = kron(eye(N), SigmaX)
G[N * K + N:, N * K + N:] = -beta * beta.T

for i in range(N):
    temp = zeros((K, K + 1))
    values = mean(u[:, i]) - multiply(beta[:, i], riskPremia)
    temp[:, 1:] = diag(values.A1)
    G[N * K + N:, i * (K + 1):(i + 1) * (K + 1)] = temp

vcv = inv(G.T) * S * inv(G) / T

In [23]:
# Chi2 test/J-value
vcvAlpha = vcv[0:N * K + N:4, 0:N * K + N:4]
J = alpha * inv(vcvAlpha) * alpha.T
J = J[0, 0]
Jpval = 1 - chi2(25).cdf(J)

# Output

In [29]:
# Output
vcvRiskPremia = vcv[N * K + N:, N * K + N:]
annualizedRP = 12 * riskPremia
arp = list(squeeze(annualizedRP.A))
arpSE = list(sqrt(12 * diag(vcvRiskPremia)))
print(' Annualized Risk Premia')
print(' Market SMB HML')
print('')
print('Premia {0:0.4f} {1:0.4f} {2:0.4f}'.format(arp[0], arp[1], arp[2]))
print('Std. Err. {0:0.4f} {1:0.4f} {2:0.4f}'.format(arpSE[0], arpSE[1], arpSE[2]))
print('\n\n')
print('test:{:0.4f}'.format(J))
print('value:{:0.4f}'.format(Jpval))
i = 0
betaSE = []
for j in range(5):
    for k in range(5):
        a = alpha[0, i]
        b = beta[:, i].A1
        variances = diag(vcv[(K + 1) * i:(K + 1) * (i + 1), (K + 1) * i:(K + 1) * (i + 1)])
        betaSE.append(sqrt(variances))
        s = sqrt(variances)
        c = hstack((a, b))
        t = c / s
        print('Size: {:}, Value:{:}   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML)'.format(j + 1, k + 1))
        print('Coefficients: {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(a, b[0], b[1], b[2]))
        print('Std Err.      {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(s[0], s[1], s[2], s[3]))
        print('Tstat         {:>10,.4f} {:>10,.4f} {:>10,.4f} {:>10,.4f}'.format(t[0], t[1], t[2], t[3]))
        print('')
        i += 1

 Annualized Risk Premia
 Market SMB HML

Premia 6.6642 2.8731 2.8080
Std. Err. 0.5994 0.4010 0.4296



test:95.2879
value:0.0000
Size: 1, Value:1   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML)
Coefficients:    -0.8354     1.3099     1.2892     0.3943
Std Err.          0.1820     0.1269     0.1671     0.2748
Tstat            -4.5904    10.3196     7.7127     1.4348

Size: 1, Value:2   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML)
Coefficients:    -0.3911     1.0853     1.6100     0.3317
Std Err.          0.1237     0.0637     0.1893     0.1444
Tstat            -3.1616    17.0351     8.5061     2.2971

Size: 1, Value:3   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML)
Coefficients:    -0.1219     1.0747     1.1812     0.4648
Std Err.          0.0997     0.0419     0.0938     0.0723
Tstat            -1.2225    25.6206    12.5952     6.4310

Size: 1, Value:4   Alpha   Beta(VWM)   Beta(SMB)   Beta(HML)
Coefficients:     0.0388     0.9630     1.2249     0.5854
Std Err.          0.0692     0.0232     0.10